# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""5398360169311843157""","""../../../../data/SampleHDS.jso…",4.0268e18,"""15892369649658543097""","""12006942628514731319""","""16148460541627308395""",false
"""11967296035907955873""","""../../../../data/SampleHDS.jso…",1.8007e19,"""2777111513117214561""","""12006942628514731319""","""14022302925660943236""",true
"""3942435161784834817""","""../../../../data/SampleHDS.jso…",1.7204e19,"""10880544421112658989""","""14378043338182894103""","""16148460541627308395""",false
"""11967296035907955873""","""../../../../data/SampleHDS.jso…",3.6445e18,"""6591241293613560564""","""13094853007860835240""","""16148460541627308395""",false
"""5398360169311843157""","""../../../../data/SampleHDS.jso…",2.7231e18,"""17600854238110156777""","""13094853007860835240""","""414425427498139499""",true
"""3942435161784834817""","""../../../../data/SampleHDS.jso…",7.9901e18,"""12713277038932338672""","""6252894449529604079""",null,false
"""3942435161784834817""","""../../../../data/SampleHDS.jso…",5.3511e18,"""4464362995733085047""","""13094853007860835240""","""414425427498139499""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""15848002248818613278""","""../../../../data/SampleHDS.jso…",1.1174e19,"""9265325719151228614""","""FirstMortgage30yr""","""17848911494475000563""","""Rejected"""
"""12137295744880477336""","""../../../../data/SampleHDS.jso…",1.7827e19,"""7786423225957336511""","""FirstMortgage30yr""","""4579887283956551099""","""Accepted"""
"""8820671509832720296""","""../../../../data/SampleHDS.jso…",1.5646e19,"""13841085348251792078""","""MoneyMarketSavingsAccount""","""17848911494475000563""","""Rejected"""
"""12137295744880477336""","""../../../../data/SampleHDS.jso…",4.6115e18,"""2677400569047684887""","""BasicChecking""","""17848911494475000563""","""Rejected"""
"""15848002248818613278""","""../../../../data/SampleHDS.jso…",8.6191e18,"""8903079026211318661""","""BasicChecking""","""16061984791637900534""","""Accepted"""
"""8820671509832720296""","""../../../../data/SampleHDS.jso…",4.6931e18,"""5274141283598386803""","""UPlusFinPersonal""",null,"""Rejected"""
"""8820671509832720296""","""../../../../data/SampleHDS.jso…",9.0406e18,"""9296877367927305653""","""BasicChecking""","""16061984791637900534""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""18275008459824845603""","""../../../../data/SampleHDS.jso…",8.0232e18,"""5649658576739015925""","""FirstMortgage30yr""","""14943934164341550875""","""Rejected"""
"""8590213773274579899""","""../../../../data/SampleHDS.jso…",2.0868e18,"""3168970876485338324""","""FirstMortgage30yr""","""2187542552179307146""","""Accepted"""
"""15363642437953389018""","""../../../../data/SampleHDS.jso…",1.5944e18,"""10049073876774012354""","""MoneyMarketSavingsAccount""","""14943934164341550875""","""Rejected"""
"""8590213773274579899""","""../../../../data/SampleHDS.jso…",1.6563e19,"""14346743418436570435""","""BasicChecking""","""14943934164341550875""","""Rejected"""
"""18275008459824845603""","""../../../../data/SampleHDS.jso…",1.7966e19,"""908489365961842223""","""BasicChecking""","""8909604001254530810""","""Accepted"""
"""15363642437953389018""","""../../../../data/SampleHDS.jso…",1.3352e19,"""16690391885718434369""","""UPlusFinPersonal""",null,"""Rejected"""
"""15363642437953389018""","""../../../../data/SampleHDS.jso…",1.2693e19,"""1214614062180410552""","""BasicChecking""","""8909604001254530810""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
filename=filename
Customer_CLV=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""5647345435870296968""",5.2020e18,"""3668836460984522919""","""FirstMortgage30yr""","""17972151441301669959""","""Rejected"""
"""11658568607905760119""",8.4144e18,"""10562454662844849814""","""FirstMortgage30yr""","""11399796510172690371""","""Accepted"""
"""1216230209140611405""",4.1117e18,"""10941993694228804024""","""MoneyMarketSavingsAccount""","""17972151441301669959""","""Rejected"""
"""11658568607905760119""",4.6745e18,"""453143593466443771""","""BasicChecking""","""17972151441301669959""","""Rejected"""
"""5647345435870296968""",3.0878e18,"""6619368202762216112""","""BasicChecking""","""565780648122698305""","""Accepted"""
"""1216230209140611405""",1.2170e18,"""8394724157197101449""","""UPlusFinPersonal""",null,"""Rejected"""
"""1216230209140611405""",6.9150e18,"""12450827582232701707""","""BasicChecking""","""565780648122698305""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'